# Sudoku-12-Z3-Python : Z3 SMT Solver (Python)**Navigation** : [<< OR-Tools Python](Sudoku-10-ORTools-Python.ipynb) | [Index](README.md) | [Symbolic Automata >>](Sudoku-13-SymbolicAutomata-Csharp.ipynb)## Objectifs d'apprentissageA la fin de ce notebook, vous saurez :1. Modéliser le Sudoku avec un solveur SMT (Z3)2. Comprendre la différence entre SAT et SMT3. Utiliser des BitVectors pour optimiser les représentations4. Comparer Z3 avec OR-Tools CP-SAT**Durée estimée** : ~10 min | **Prérequis** : [Sudoku-0 Environment](Sudoku-0-Environment-Csharp.ipynb)---Ce notebook implémente un solveur de Sudoku utilisant **Microsoft Z3**, un solveur **SMT** (Satisfiability Modulo Theories).## Installation

In [1]:
# Imports
import time
from typing import List, Optional, Tuple

# Vérifier les installations
try:
    from ortools.sat.python import cp_model
    print(f"OR-Tools importé avec succès")
except ImportError:
    print("OR-Tools non installé. Exécutez: pip install ortools")

try:
    from z3 import *
    print(f"Z3 importé avec succès")
except ImportError:
    print("Z3 non installé. Exécutez: pip install z3-solver")

OR-Tools importé avec succès
Z3 importé avec succès


## 1. Solveur Z3 SMT**Microsoft Z3** est un solveur **SMT** (Satisfiability Modulo Theories) - il combine la puissance des solveurs SAT avec des théories mathématiques (arithmétique, tableaux, bitvectors, etc.).

### Différence entre SAT et SMT| SAT | SMT ||-----|-----|| Variables booléennes uniquement | Variables typées (Int, Real, BitVec, Array...) || Clauses propositionnelles | Formules de premier ordre || `(x OR y) AND (NOT x OR z)` | `x + y > 10 AND x < 5` |

## 1. Configuration du chemin vers les puzzles

In [3]:
class SudokuGrid:
    """Représentation d'une grille de Sudoku 9x9."""
    
    def __init__(self, grid: Optional[List[List[int]]] = None):
        if grid is None:
            self.cells = [[0] * 9 for _ in range(9)]
        else:
            self.cells = [row[:] for row in grid]
    
    @classmethod
    def from_string(cls, s: str) -> 'SudokuGrid':
        s = s.replace('.', '0').replace(' ', '').replace('\n', '')
        if len(s) != 81:
            raise ValueError(f"La chaîne doit avoir 81 caractères")
        grid = cls()
        for i in range(81):
            grid.cells[i // 9][i % 9] = int(s[i])
        return grid
    
    def clone(self) -> 'SudokuGrid':
        return SudokuGrid(self.cells)
    
    def __str__(self) -> str:
        lines = []
        for r in range(9):
            if r > 0 and r % 3 == 0:
                lines.append('-' * 21)
            row_str = ''
            for c in range(9):
                if c > 0 and c % 3 == 0:
                    row_str += '| '
                val = self.cells[r][c]
                row_str += (str(val) if val != 0 else '.') + ' '
            lines.append(row_str)
        return '\n'.join(lines)

def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:
    puzzles = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) >= 81:
                puzzles.append(line[:81])
                if max_puzzles and len(puzzles) >= max_puzzles:
                    break
    return puzzles

# Charger puzzles
easy_puzzles = load_puzzles(str(PUZZLES_DIR / 'Sudoku_Easy51.txt'), max_puzzles=10)
hard_puzzles = load_puzzles(str(PUZZLES_DIR / 'Sudoku_hardest.txt'))
print(f"Puzzles chargés: {len(easy_puzzles)} faciles, {len(hard_puzzles)} difficiles")

Puzzles chargés: 10 faciles, 11 difficiles


In [5]:
class Z3Solver:
    """Solveur Sudoku utilisant Z3 SMT."""
    
    def solve(self, grid: SudokuGrid) -> bool:
        """Résout le Sudoku avec Z3."""
        # Créer les variables: X[i][j] est un entier 1-9
        X = [[Int(f'x_{i}_{j}') for j in range(9)] for i in range(9)]
        
        solver = Solver()
        
        # Contraintes de domaine: 1 <= X[i][j] <= 9
        for i in range(9):
            for j in range(9):
                solver.add(X[i][j] >= 1, X[i][j] <= 9)
        
        # Valeurs initiales
        for i in range(9):
            for j in range(9):
                if grid.cells[i][j] != 0:
                    solver.add(X[i][j] == grid.cells[i][j])
        
        # Contraintes: toutes différentes par ligne
        for i in range(9):
            solver.add(Distinct([X[i][j] for j in range(9)]))
        
        # Contraintes: toutes différentes par colonne
        for j in range(9):
            solver.add(Distinct([X[i][j] for i in range(9)]))
        
        # Contraintes: toutes différentes par bloc 3x3
        for box_row in range(3):
            for box_col in range(3):
                box_cells = []
                for i in range(3):
                    for j in range(3):
                        box_cells.append(X[box_row * 3 + i][box_col * 3 + j])
                solver.add(Distinct(box_cells))
        
        # Résoudre
        if solver.check() == sat:
            model = solver.model()
            for i in range(9):
                for j in range(9):
                    grid.cells[i][j] = model.evaluate(X[i][j]).as_long()
            return True
        
        return False

# Test
z3_solver = Z3Solver()
test_grid = SudokuGrid.from_string(hard_puzzles[0])
print("Puzzle difficile:")
print(test_grid)

start = time.time()
solved = z3_solver.solve(test_grid)
elapsed = (time.time() - start) * 1000

print(f"\nRésolu: {solved} en {elapsed:.2f} ms")
print("\nSolution:")
print(test_grid)

Puzzle difficile:
8 5 . | . . 2 | 4 . . 
7 2 . | . . . | . . 9 
. . 4 | . . . | . . . 
---------------------
. . . | 1 . 7 | . . 2 
3 . 5 | . . . | 9 . . 
. 4 . | . . . | . . . 
---------------------
. . . | . 8 . | . 7 . 
. 1 7 | . . . | . . . 
. . . | . 3 6 | . 4 . 



Résolu: True en 505.67 ms

Solution:
8 5 9 | 6 1 2 | 4 3 7 
7 2 3 | 8 5 4 | 1 6 9 
1 6 4 | 3 7 9 | 5 2 8 
---------------------
9 8 6 | 1 4 7 | 3 5 2 
3 7 5 | 2 6 8 | 9 1 4 
2 4 1 | 5 9 3 | 7 8 6 
---------------------
4 3 2 | 9 8 1 | 6 7 5 
6 1 7 | 4 2 5 | 8 9 3 
5 9 8 | 7 3 6 | 2 4 1 


In [7]:
class Z3BitVectorSolver:
    """Solveur Z3 utilisant des BitVectors 4 bits."""
    
    def solve(self, grid: SudokuGrid) -> bool:
        """Résout avec BitVectors."""
        # BitVectors 4 bits (valeurs 1-9 tiennent dans 4 bits)
        X = [[BitVec(f'x_{i}_{j}', 4) for j in range(9)] for i in range(9)]
        
        solver = Solver()
        
        # Contraintes de domaine
        for i in range(9):
            for j in range(9):
                solver.add(UGE(X[i][j], 1))  # >= 1 (unsigned)
                solver.add(ULE(X[i][j], 9))  # <= 9 (unsigned)
        
        # Valeurs initiales
        for i in range(9):
            for j in range(9):
                if grid.cells[i][j] != 0:
                    solver.add(X[i][j] == grid.cells[i][j])
        
        # Contraintes Distinct (par paires pour BitVec)
        def add_all_different(cells):
            for i in range(len(cells)):
                for j in range(i + 1, len(cells)):
                    solver.add(cells[i] != cells[j])
        
        # Lignes
        for i in range(9):
            add_all_different([X[i][j] for j in range(9)])
        
        # Colonnes
        for j in range(9):
            add_all_different([X[i][j] for i in range(9)])
        
        # Blocs
        for box_row in range(3):
            for box_col in range(3):
                box_cells = [X[box_row * 3 + i][box_col * 3 + j] 
                             for i in range(3) for j in range(3)]
                add_all_different(box_cells)
        
        if solver.check() == sat:
            model = solver.model()
            for i in range(9):
                for j in range(9):
                    grid.cells[i][j] = model.evaluate(X[i][j]).as_long()
            return True
        
        return False

# Comparaison Z3 Int vs BitVec
print("=== Comparaison Z3 Int vs BitVector ===")

z3_int = Z3Solver()
z3_bv = Z3BitVectorSolver()

for i, puzzle_str in enumerate(hard_puzzles[:5]):
    print(f"\nPuzzle {i+1}:")
    
    grid1 = SudokuGrid.from_string(puzzle_str)
    start = time.time()
    z3_int.solve(grid1)
    t1 = (time.time() - start) * 1000
    
    grid2 = SudokuGrid.from_string(puzzle_str)
    start = time.time()
    z3_bv.solve(grid2)
    t2 = (time.time() - start) * 1000
    
    print(f"  Z3 Int:      {t1:>8.2f} ms")
    print(f"  Z3 BitVec:   {t2:>8.2f} ms")

=== Comparaison Z3 Int vs BitVector ===

Puzzle 1:


  Z3 Int:        261.39 ms
  Z3 BitVec:     119.00 ms

Puzzle 2:


  Z3 Int:        472.56 ms
  Z3 BitVec:      70.92 ms

Puzzle 3:


  Z3 Int:        160.51 ms
  Z3 BitVec:      54.34 ms

Puzzle 4:


  Z3 Int:        348.04 ms
  Z3 BitVec:      66.19 ms

Puzzle 5:
  Z3 Int:         95.62 ms
  Z3 BitVec:      62.37 ms


## 6. Résumé et conclusions

### Comparaison des approches

| Solveur | Type | Avantages | Inconvénients |
|---------|------|-----------|---------------|
| **Backtracking** | Impératif | Simple, pédagogique | Lent sur puzzles difficiles |
| **Backtracking+MRV** | Impératif | Bon compromis | Heuristique manuelle |
| **OR-Tools CP-SAT** | Déclaratif | Très rapide, API claire | Dépendance externe |
| **Z3 Int** | Déclaratif | Flexible, expressif | Plus lent que CP-SAT |
| **Z3 BitVec** | Déclaratif | Optimisé bas niveau | Syntaxe plus complexe |

### Recommandations par cas d'usage

- **Apprentissage des algorithmes**: Commencer par le backtracking simple
- **Production / Performance**: OR-Tools CP-SAT
- **Recherche / Contraintes complexes**: Z3
- **Problèmes combinatoires variés**: OR-Tools (portfolio de solveurs)

### Au-delà du Sudoku

Ces solveurs peuvent résoudre une grande variété de problèmes:
- **OR-Tools**: Planification, routage, ordonnancement, bin packing
- **Z3**: Vérification de programmes, analyse de sécurité, synthèse de code

Le Sudoku est un excellent problème pédagogique car il est facile à comprendre mais assez complexe pour illustrer les concepts clés de la programmation par contraintes.

## Navigation

### Notebooks Python de cette serie
- [Sudoku-Python-Backtracking.ipynb](Sudoku-Python-Backtracking.ipynb) - Backtracking et MRV
- **Ce notebook**: OR-Tools CP-SAT et Z3 SMT
- [Sudoku-Python-Genetic.ipynb](Sudoku-Python-Genetic.ipynb) - Algorithmes genetiques (PyGAD)
- [Sudoku-Python-DancingLinks.ipynb](Sudoku-Python-DancingLinks.ipynb) - Algorithm X / Dancing Links

### Notebooks C# equivalents
- [Sudoku-3-ORTools.ipynb](Sudoku-3-ORTools.ipynb) - OR-Tools en C#
- [Sudoku-4-Z3.ipynb](Sudoku-4-Z3.ipynb) - Z3 en C#

---
**Navigation** : [<< Python Backtracking](Sudoku-Python-Backtracking.ipynb) | [Index](README.md) | [Python Genetic >>](Sudoku-Python-Genetic.ipynb)